In [ ]:
from google.colab import drive
drive.mount('/content/drive')

!mkdir -p ~/.kaggle
!cp "/content/drive/MyDrive/kaggle.json" ~/.kaggle/kaggle.json
!chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets download -d adelfr2009/ip102-new --unzip --force # download data

Mounted at /content/drive
Dataset URL: https://www.kaggle.com/datasets/adelfr2009/ip102-new
License(s): unknown
 98% 3.63G/3.70G [00:10<00:00, 528MB/s]
100% 3.70G/3.70G [00:10<00:00, 364MB/s]


In [ ]:
import os
import shutil
from tqdm import tqdm

# Paths
base_dir = "/content/ip102/Classification"
images_dir = os.path.join(base_dir, "images")
output_dir = "/content/ip102dataset"

# Include test split
splits = {
    "train": os.path.join(base_dir, "train.txt"),
    "val": os.path.join(base_dir, "val.txt"),
    "test": os.path.join(base_dir, "test.txt"),
}

# Process each split
for split, split_file in splits.items():
    if not os.path.exists(split_file):
        print(f"Missing file: {split_file}")
        continue

    with open(split_file, "r") as f:
        lines = f.readlines()

    print(f"Processing {split} set ({len(lines)} images)...")

    for line in tqdm(lines, desc=f"Copying {split}"):
        img_name, class_id = line.strip().split()
        src = os.path.join(images_dir, img_name)
        dst_dir = os.path.join(output_dir, split, f"class{class_id}")
        dst = os.path.join(dst_dir, img_name)

        os.makedirs(dst_dir, exist_ok=True)

        if os.path.exists(src):
            shutil.copy2(src, dst)
        else:
            print(f"Image not found: {src}")


Processing train set (45095 images)...


Copying train: 100%|██████████| 45095/45095 [00:06<00:00, 7076.15it/s]


Processing val set (7508 images)...


Copying val: 100%|██████████| 7508/7508 [00:01<00:00, 7347.42it/s]


Processing test set (22619 images)...


Copying test: 100%|██████████| 22619/22619 [00:03<00:00, 7456.56it/s]


In [ ]:
import argparse
import time
import yaml
import os
import glob
import math
import logging
from collections import OrderedDict
from contextlib import suppress
from datetime import datetime

import torch
import torch.nn as nn
import torchvision.utils
from torch.nn.parallel import DistributedDataParallel as NativeDDP

from timm.data import (
    create_dataset,
    create_loader,
    resolve_data_config,
    Mixup,
    FastCollateMixup,
    AugMixDataset,
)
from timm.models import (
    create_model,
    safe_model_name,
    resume_checkpoint,
    load_checkpoint,
    model_parameters,
)
from timm.layers import convert_splitbn_model
from timm.utils import *
from timm.loss import *
from timm.optim import create_optimizer_v2, optimizer_kwargs
from timm.scheduler import create_scheduler
from timm.utils import ApexScaler, NativeScaler

#from misc.distillation_loss import DistillationLoss
#from misc.cosine_annealing import CosineWDSchedule

In [ ]:
!pip install onnx

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 117.0 MB/s eta 0:00:00


In [ ]:
import torch
from timm.models import create_model

# Step 1: Create the same model architecture
model = create_model(
    "fastvit_sa12",
    pretrained=False,
    num_classes=102,
)



# Step 2: Load your trained checkpoint
checkpoint = torch.load('/content/drive/MyDrive/fastvit_sa12.pth.tar')
model.load_state_dict(checkpoint['state_dict'])

#model_inf = reparameterize_model(model)

<All keys matched successfully>

In [ ]:
# Export the model
x = torch.randn(1, 3, 224, 224, requires_grad=True)

torch.onnx.export(model,              # model being run
                  x,                         # model input (or a tuple for multiple inputs)
                  "fastvit.onnx", # where to save the model (can be a file or file-like object)
                  export_params=True,        # store the trained parameter weights inside the model file
                  opset_version=14,          # the ONNX version to export the model to
                  do_constant_folding=True,  # whether to execute constant folding for optimization
                  input_names = ['input'],   # the model's input names
                  output_names = ['output']) # the model's output names